In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec
from lark.data import *

1.8.1
0.8.0a0+e4e171a


In [141]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [156]:
cfg = Config(
    noise_nsr_dbs = [30, 35],
    sites=['SSW'], 
    use_neptune=True, 
    log_batch_metrics=False,
    n_epochs=250, 
    bs=32, 
    lr=1e-3, 
    model='resnet18',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [157]:
cfg.training_dataset_size

21504

as suggested here:
https://stackoverflow.com/a/47212899/478746

In [158]:
main_model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
for param in main_model.parameters():
    param.requires_grad = False

for layer in [main_model.layer3, main_model.layer4, main_model.avgpool]:
    for param in layer.parameters():
        param.requires_grad = True

prep = MixedSig2Spec(cfg)
posp = torch.nn.Linear(in_features=512, out_features=len(cfg.labels), bias=True)
main_model.fc = posp

model = torch.nn.Sequential(prep, main_model)
model = model.cuda()

Using cache found in /home/koen/.cache/torch/hub/pytorch_vision_v0.9.0


In [159]:
lrn = Learner("resnet18-half-frozen", cfg, model)

resnet18-half-frozen-20210507-214000


/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['secondary_labels'] = self.df_meta['secondary_labels'].str.replace("[\[\]',]", '').str.split()
/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:77: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['filename'] = self.df_meta['filename'].str.replace(".ogg", '.wav')


In [ ]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-102


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

2021-05-07 22:45:27 epoch:  45 train loss: 0.191955 train f1: 0.349089 valid loss: 0.170411 valid f1: 0.079614


  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-07 22:46:53 epoch:  46 train loss: 0.188798 train f1: 0.354608 valid loss: 0.166973 valid f1: 0.097052


  0%|          | 0/672 [00:00<?, ?it/s]

In [132]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,432,28534,4296,338,0.157148
2,0.2,301,31509,1321,469,0.251672
3,0.3,229,32156,674,541,0.273760
4,0.4,186,32440,390,584,0.276374
5,0.5,151,32584,246,619,0.258783
6,0.6,126,32666,164,644,0.237736
7,0.7,101,32714,116,669,0.204661
8,0.8,78,32771,59,692,0.171996
9,0.9,53,32796,34,717,0.123687


In [133]:
lrn.load_checkpoint('best')

{'epoch': 22,
 'valid_loss': 0.1087923233250254,
 'valid_score': 0.2665489912033081}

In [134]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,441,28642,4188,329,0.163364
2,0.2,302,31612,1218,468,0.263755
3,0.3,225,32227,603,545,0.281602
4,0.4,183,32489,341,587,0.282844
5,0.5,151,32618,212,619,0.266549
6,0.6,122,32691,139,648,0.236663
7,0.7,99,32741,89,671,0.206681
8,0.8,79,32785,45,691,0.176734
9,0.9,53,32812,18,717,0.126040


In [135]:
lrn.scheduler.state_dict()

{'total_steps': 25200,
 '_schedule_phases': [{'end_step': 7559.0,
   'start_lr': 'initial_lr',
   'end_lr': 'max_lr',
   'start_momentum': 'max_momentum',
   'end_momentum': 'base_momentum'},
  {'end_step': 25199,
   'start_lr': 'max_lr',
   'end_lr': 'min_lr',
   'start_momentum': 'base_momentum',
   'end_momentum': 'max_momentum'}],
 'anneal_func': <bound method OneCycleLR._annealing_cos of <torch.optim.lr_scheduler.OneCycleLR object at 0x7f8f4a0c0250>>,
 'cycle_momentum': True,
 'use_beta1': True,
 'base_lrs': [0.0004],
 'last_epoch': 23184,
 '_step_count': 23185,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0003185515568067364]}

In [136]:
for param in model.parameters():
    param.requires_grad = True

In [138]:
cfg = Config(
    noise_nsr_dbs = [30],
    sites=['SSW'], 
    use_neptune=True, 
    log_batch_metrics=True,
    n_epochs=100, 
    bs=32, 
    lr=1e-4,
    model='resnet18',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [139]:
lrn = Learner("resnet18-unfrozen", cfg, model)

resnet18-unfrozen-20210507-200950


/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['secondary_labels'] = self.df_meta['secondary_labels'].str.replace("[\[\]',]", '').str.split()
/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:77: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['filename'] = self.df_meta['filename'].str.replace(".ogg", '.wav')


In [140]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-99


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/1008 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-07 21:20:14 epoch:  33 train loss: 0.173344 train f1: 0.437608 valid loss: 0.108681 valid f1: 0.276857


  0%|          | 0/1008 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [110]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,375,29302,3528,395,0.160496
2,0.2,220,31955,875,550,0.235925
3,0.3,156,32391,439,614,0.228571
4,0.4,130,32553,277,640,0.220901
5,0.5,109,32657,173,661,0.207224
6,0.6,90,32707,123,680,0.183113
7,0.7,71,32755,75,699,0.155022
8,0.8,58,32786,44,712,0.133028
9,0.9,33,32814,16,737,0.080586


In [111]:
lrn.load_checkpoint('best')
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,380,29322,3508,390,0.163160
2,0.2,223,31959,871,547,0.239270
3,0.3,161,32395,435,609,0.235725
4,0.4,133,32557,273,637,0.226190
5,0.5,112,32644,186,658,0.209738
6,0.6,96,32708,122,674,0.194332
7,0.7,66,32749,81,704,0.143948
8,0.8,56,32781,49,714,0.128000
9,0.9,36,32815,15,734,0.087698


In [114]:
lrn.load_checkpoint('latest')

{'epoch': 9,
 'valid_loss': 0.11422962146370035,
 'valid_score': 0.20722433924674988}

In [115]:
lrn.scheduler.state_dict()

{'total_steps': 3360,
 '_schedule_phases': [{'end_step': 1007.0,
   'start_lr': 'initial_lr',
   'end_lr': 'max_lr',
   'start_momentum': 'max_momentum',
   'end_momentum': 'base_momentum'},
  {'end_step': 3359,
   'start_lr': 'max_lr',
   'end_lr': 'min_lr',
   'start_momentum': 'base_momentum',
   'end_momentum': 'max_momentum'}],
 'anneal_func': <bound method OneCycleLR._annealing_cos of <torch.optim.lr_scheduler.OneCycleLR object at 0x7f8fd6290760>>,
 'cycle_momentum': True,
 'use_beta1': True,
 'base_lrs': [4e-06],
 'last_epoch': 3360,
 '_step_count': 3361,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.446029226779303e-10]}

In [116]:
cfg = Config(
    noise_nsr_dbs = [35],
    sites=['SSW'], 
    use_neptune=True, 
    log_batch_metrics=True,
    n_epochs=100, 
    bs=32, 
    lr=1e-5,
    model='resnet18',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [117]:
lrn = Learner("resnet18-unfrozen", cfg, model)

resnet18-unfrozen-20210507-162529


/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['secondary_labels'] = self.df_meta['secondary_labels'].str.replace("[\[\]',]", '').str.split()
/home/koen/projects/botkop/lark/nbs/botkop/lark/data.py:77: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_meta['filename'] = self.df_meta['filename'].str.replace(".ogg", '.wav')


In [118]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-96


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-07 17:34:19 epoch:  83 train loss: 0.208482 train f1: 0.228448 valid loss: 0.116329 valid f1: 0.198473


  0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 